<a href="https://colab.research.google.com/github/aniparnagian/Artsakh_War_Sentiment-and_LDA_Project-/blob/main/Gensim_Tweets_Artsakh_War.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [1]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 8.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=ea23537e6ce9b458af7e295e960c0e94dd1661f1de341f51de933e45b8b50a86
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=133a30016ca5ae23dfccb5773e9526032d046886d95d1066bd6eaee9b2031aaa
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [3]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 7.3 MB/s 
     |████████████████████████████████| 110 kB 51.3 MB/s 


## Configure Jupyter Notebook

In [4]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [6]:
from slugify import slugify

# Get Text Data


## Upload Raw Textfile

In [7]:
from google.colab import files

TEXT_ENCODING = 'utf-8'

In [8]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

Saving tweets_raw_artsakhwar_20201028_145527_20221214_195302.csv to tweets_raw_artsakhwar_20201028_145527_20221214_195302.csv
User uploaded file "tweets_raw_artsakhwar_20201028_145527_20221214_195302.csv" with length 9680781 bytes
CPU times: user 1.35 s, sys: 185 ms, total: 1.54 s
Wall time: 1min 6s


In [9]:
# Recover parameters of original search from filename components
word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')
end_time = ''.join(end_time.split('.')[:-1])
end_time = ''.join(end_time.split('(')[0])

print(f'word1: {word1}')
print(f'word2: {word2}')
print(f'search_str: {search_str}')
print(f'start_date: {start_date}')
print(f'start_time: {start_time}')
print(f'end_date: {end_date}')
print(f'end_time: {end_time}')

word1: tweets
word2: raw
search_str: artsakhwar
start_date: 20201028
start_time: 145527
end_date: 20221214
end_time: 195302


In [10]:
# Read raw tweet datafile into DataFrame 'data'

try:
  headers = ['datetime','id','tweet','username','reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id',
             'text_raw','text_noemoji','text_clean','vader','textblob','distilbert'] # ,'roberta']
  dtypes = {'datetime':'str',
            'id':'int',
            'tweet':'string',
            'username':'string',
            'reply_ct':'int',
            'retweet_ct':'int',
            'like_ct':'int',
            'quote_ct':'int',
            'retweeted_id':'string',
            'quoted_id':'string',
            'text_raw':'string',
            'text_noemoji':'string',
            'text_clean':'string',
            'vader':'float',
            'textblob':'float',
            'distilbert':'float'}
            # 'roberta':'float'}
  parse_dates = ['datetime']
  # text_df = pd.read_csv(text_filename_str, skiprows=[0], names=headers, dtype=dtypes, parse_dates=parse_dates, index_col=None)
  text_df = pd.read_csv(text_filename_str, index_col=None)
  text_df.head()
  text_df.info()
except:
  print(f'ERROR: Tweets datafile cannot be read by pd.read_csv')

,datetime,id,tweet,username,reply_ct,retweet_ct,like_ct,quote_ct,retweeted_id,quoted_id
0,2020-10-28 14:55:27+00:00,1321465607242412035,End this war. Azerbaijan military bombed the M...,SophiaArmen,1,48,77,7,NaN,NaN
1,2020-10-28 14:55:32+00:00,1321465630130761728,People of Artsakh are Azerbaijan’s target – Ar...,TrueNewsArmenia,0,0,0,0,NaN,NaN
2,2020-10-28 14:56:45+00:00,1321465936252121090,"This war crime, which is a gross violation of ...",Gohar55075973,0,0,0,0,NaN,NaN
3,2020-10-28 14:56:48+00:00,1321465948658806786,People of Artsakh are Azerbaijan’s target – Ar...,hasmikisaghulya,0,0,0,0,NaN,NaN
4,2020-10-28 14:57:40+00:00,1321466165177233408,Armenia want peace❤️💙🧡\nThe war in Artsakh was...,AnnaZohranyan2,0,1,4,0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      30000 non-null  object 
 1   id            30000 non-null  int64  
 2   tweet         30000 non-null  object 
 3   username      30000 non-null  object 
 4   reply_ct      30000 non-null  int64  
 5   retweet_ct    30000 non-null  int64  
 6   like_ct       30000 non-null  int64  
 7   quote_ct      30000 non-null  int64  
 8   retweeted_id  0 non-null      float64
 9   quoted_id     4926 non-null   object 
dtypes: float64(1), int64(5), object(4)
memory usage: 2.3+ MB


## Clean DataFrame

In [11]:
# Drop unused columns

if 'reply_ct' in text_df.columns:
  text_df.drop(columns=['reply_ct'], inplace=True)

if 'retweet_ct' in text_df.columns:
  text_df.drop(columns=['retweet_ct'], inplace=True)

if 'like_ct' in text_df.columns:
  text_df.drop(columns=['like_ct'], inplace=True)

if 'quote_ct' in text_df.columns:
  text_df.drop(columns=['quote_ct'], inplace=True)

if 'retweeted_id' in text_df.columns:
  text_df.drop(columns=['retweeted_id'], inplace=True)

if 'quoted_id' in text_df.columns:
  text_df.drop(columns=['quoted_id'], inplace=True)


In [12]:
# Clean and drop illegal datetime values

text_df['datetime'] = pd.to_datetime(text_df['datetime'], errors='coerce')
text_df = text_df.dropna(subset=['datetime'])
text_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  30000 non-null  datetime64[ns, UTC]
 1   id        30000 non-null  int64              
 2   tweet     30000 non-null  object             
 3   username  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ MB


In [13]:
# Drop any row that have at any NaN

text_df.dropna(inplace=True)
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  30000 non-null  datetime64[ns, UTC]
 1   id        30000 non-null  int64              
 2   tweet     30000 non-null  object             
 3   username  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
text_df['text_raw'] = text_df['tweet']

text_df['text_raw_type'] = text_df['text_raw'].apply(lambda x: type(x))
text_df['text_raw_type'].value_counts()

<class 'str'>    30000
Name: text_raw_type, dtype: int64

In [15]:
text_df.convert_dtypes()
text_df.info()

,datetime,id,tweet,username,text_raw,text_raw_type
0,2020-10-28 14:55:27+00:00,1321465607242412035,End this war. Azerbaijan military bombed the M...,SophiaArmen,End this war. Azerbaijan military bombed the M...,<class 'str'>
1,2020-10-28 14:55:32+00:00,1321465630130761728,People of Artsakh are Azerbaijan’s target – Ar...,TrueNewsArmenia,People of Artsakh are Azerbaijan’s target – Ar...,<class 'str'>
2,2020-10-28 14:56:45+00:00,1321465936252121090,"This war crime, which is a gross violation of ...",Gohar55075973,"This war crime, which is a gross violation of ...",<class 'str'>
3,2020-10-28 14:56:48+00:00,1321465948658806786,People of Artsakh are Azerbaijan’s target – Ar...,hasmikisaghulya,People of Artsakh are Azerbaijan’s target – Ar...,<class 'str'>
4,2020-10-28 14:57:40+00:00,1321466165177233408,Armenia want peace❤️💙🧡 The war in Artsakh was ...,AnnaZohranyan2,Armenia want peace❤️💙🧡 The war in Artsakh was ...,<class 'str'>
...,...,...,...,...,...,...
29995,2022-12-14 18:28:01+00:00,1603094474913103872,"In an op-ed for @Newsweek, human rights lawyer...",agbu,"In an op-ed for @Newsweek, human rights lawyer...",<class 'str'>
29996,2022-12-14 18:41:01+00:00,1603097748030595074,#SaveArtsakh 🇦🇲❤️ #Armenia #Artsakh #War #Ar...,PatmakanHetq,#SaveArtsakh 🇦🇲❤️ #Armenia #Artsakh #War #Ar...,<class 'str'>
29997,2022-12-14 19:11:36+00:00,1603105443072847872,I found more photos of the fascist war profite...,realAniHagopian,I found more photos of the fascist war profite...,<class 'str'>
29998,2022-12-14 19:21:00+00:00,1603107807661735938,Commemorating the 2020 Artsakh War in LA's Arm...,donnakaz,Commemorating the 2020 Artsakh War in LA's Arm...,<class 'str'>


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       30000 non-null  datetime64[ns, UTC]
 1   id             30000 non-null  int64              
 2   tweet          30000 non-null  object             
 3   username       30000 non-null  object             
 4   text_raw       30000 non-null  object             
 5   text_raw_type  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 1.6+ MB


In [16]:
# Try to convert Tweet text to string (raise error if incompatible encoding)

# text_df['text_raw'] = text_df['text_raw'].apply(lambda x: str(x)) # 
# text_df[['text_raw','username','tweet','text_noemoji','text_clean']] = text_df[['text_raw','username','tweet','text_noemoji','text_clean']].astype('string', errors='raise') # , errors='coerce')
text_df[['text_raw','username','tweet']] = text_df[['text_raw','username','tweet']].astype('string', errors='raise') # , errors='coerce')
# text_df = text_df.dropna(subset=['text_raw'])
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       30000 non-null  datetime64[ns, UTC]
 1   id             30000 non-null  int64              
 2   tweet          30000 non-null  string             
 3   username       30000 non-null  string             
 4   text_raw       30000 non-null  string             
 5   text_raw_type  30000 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(1), string(3)
memory usage: 1.6+ MB


In [17]:
line_ls = text_df.text_raw.to_list()
print(f'BEFORE filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if isinstance(x, str)]
# print(f' AFTER filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if len(x) > 0]
# print(f' AFTER filtering out null lines: len(line_ls): {len(line_ls)}')

BEFORE filtering out NAType: len(line_ls): 30000


In [18]:
# Agglomerate n Tweets into a Synthetic Document for Topic Modeling

doc_ls = []
tweets_per_doc = 10
doc_current_ls = []
j = 0

for i, aline in enumerate(line_ls):
  
  if (((i % tweets_per_doc) == 0) & (i>0)):
    doc_temp_str = '\n'.join(doc_current_ls)
    doc_ls.append(doc_temp_str)
    print(f'Synthetic Doc #{j}: {doc_temp_str}\n\n')
    j += 1
    # Reset current syn doc
    doc_current_ls = []
    doc_current_ls.append(aline)

  else:
    try:
      doc_current_ls.append(aline)
    except:
      print(f'ERROR: Cannot strip()')

  # line_doc_ls.append(aline)

print('\n\n')

text_raw_str = "\n\n".join(doc_ls)
text_raw_str[:5]


Streaming output truncated to the last 5000 lines.
https://t.co/UNLIvnm5cq
@ArmineArutyuny1 @VRSoloviev Wake up,Armenians of World!Azeris liberate their lands;leaders of Armenia hiding behind Russian soldiers force Armenians to the war in foreign lands,use slogan Artsakh to seize power,money,handouts from diaspora.Roots of the war: 
https://t.co/8EYTdX5iDB

https://t.co/UNLIvnm5cq
@VRSoloviev Wake up,Armenians of World!Azeris liberate their lands;leaders of Armenia hiding behind Russian soldiers force Armenians to the war in foreign lands,use slogan Artsakh to seize power,money,handouts from diaspora.Roots of the war: 
https://t.co/8EYTdX5iDB

https://t.co/UNLIvnm5cq
What happened in Artsakh needs to be condemned. In the civilized 21st century countries SHOULD NOT get away with cold blooded murder and violations of war regulations #SanctionAzerbaijan #SanctionTurkey https://t.co/FfJvc3VBnm
@NikolPashinyan Wake up,Armenians of World!Azeris liberate their lands;leaders of Armenia hiding 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
doc_ls[-1]
len(doc_ls)

'@Melmouth1 @AvetissianAn Its Pashinyan and definitly not Azerbaijan who is actually blocking Artsakh and threatens to start another war\nPresident Bako Sahakyan partook in the capital Stepanakert at a solemn ceremony of opening the monument of the active participant of the Artsakh Liberation War, former head of the NKR Defense Army Staff, lieutenant-general Anatoly Zinevich.⤵️https://t.co/FaiClje2qH\n@EmmanuelMacron Will you be tweeting and standing for Artsakh as well? 2 full days of blockage and counting yet no one seems to care.\nBeing Armenian came up on a first date recently in relation to NK and the 2020 war (I’m fun on dates. Why do you ask?). Imagine my surprise when he said “Artsakh” and delved into it. A reminder: it’s almost 2023, don’t tell me you don’t know. Don’t pretend apathy for ignorance.\nThe Azerbaijani "eco-activists" trapping 120k in Artsakh with no gas have just released 44 doves in the air as a "symbol of peace." \n\n44 is the number of days the war Azerbaijan 

2999

# Enter Title

In [20]:
# If this is a Twitter textfile, the search terms are recovered from the filename

# Recover parameters of original search from filename components
try:
  word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

  print(f'word1: {word1}')
  print(f'word2: {word2}')
  print(f'search_str: {search_str}')
  print(f'start_date: {start_date}')
  print(f'end_date: {end_date}')
  print('\n')
  
  text_title_str = f'[{search_str}] from [{start_date}] to [{end_date}]'
  print(f'\nTwitter Search String:\n\n  {text_title_str}\n')

except:
  print('This is not a Twitter datafile named in standard format\n\nEnter the Novel information below.')

word1: tweets
word2: raw
search_str: artsakhwar
start_date: 20201028
end_date: 20221214



Twitter Search String:

  [artsakhwar] from [20201028] to [20221214]



In [21]:
#@title Enter Title "[Title] by [Author]""

Text_Title = "  [artsakhwar] from [20201028] to [20221214]" #@param {type:"string"}

# Remove illegal filename punctuation
Text_Title = Text_Title.replace("'","")
Text_Title = Text_Title.replace('"','')
Text_Title = Text_Title.replace('.',' ')

text_title_str = ' '.join([x.capitalize() for x in slugify(Text_Title).replace('-',' ').split()])
text_title_str

'Artsakhwar From 20201028 To 20221214'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Segment Text

In [22]:
text_raw_str[:500]
print('\n')
len(text_raw_str)

'End this war. Azerbaijan military bombed the Maternity Hospital in Stepanakert #Artsakh #NagornoKarabakh https://t.co/2aF8Nq1Vhm\nPeople of Artsakh are Azerbaijan’s target – Armenian MFA voices about war crimes https://t.co/pPiXAY1FyM\nThis war crime, which is a gross violation of international humanitarian law and customary law, clearly demonstrates that the people of Artsakh are Azerbaijan’s target, including newborn babies, mothers, and the elders.\n#StopAzerbaijaniAggerssion https://t.co/7kn7uB'

7173113

In [23]:
"""
# Split into paragraphs
text_parags_ls = text_raw_str.split('\n')
print(f'There are {len(text_parags_ls)} raw paragraphs\n')

# Delete empty paragraphs
text_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]
print(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')

# Replace stray/embedded /n with a space
text_parags_clean_ls = []
for aparag in text_parags_ls:
  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())

text_parags_ls = text_parags_clean_ls
text_parags_ls[:5] 
"""

text_parags_ls = doc_ls

"\n# Split into paragraphs\ntext_parags_ls = text_raw_str.split('\n')\nprint(f'There are {len(text_parags_ls)} raw paragraphs\n')\n\n# Delete empty paragraphs\ntext_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]\nprint(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')\n\n# Replace stray/embedded /n with a space\ntext_parags_clean_ls = []\nfor aparag in text_parags_ls:\n  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())\n\ntext_parags_ls = text_parags_clean_ls\ntext_parags_ls[:5] \n"

In [24]:
# Verify first paragraph (after title and chapter headings)

text_parags_ls[3]
print('\n')
# Paragraph char count
len(text_parags_ls[3])

"#Artsakh There is children in the artillery unit of armenia army but armenians doesn't care about it and think they still winning this war \nwhen this kid killed by azerbaijan army they accuse azerbaijan to killing childrens \n\nwhat a shame. https://t.co/f0hxZDtDUf\n#Azerbaijan is committed war crime @UNICEF @UN #StopAzerbaijaniAggression #stopterrorism in #Artsakh https://t.co/OvTUXZtLs7\nThis war crime, which is a gross violation of international humanitarian law and customary law, clearly demonstrates that the people of Artsakh are Azerbaijan’s target, including newborn babies, mothers.\n@SecPompeo @KremlinRussia @EmmanuelMacron https://t.co/bCl6Ep4YFk\n@ckouzou @Pres_Artsakh If they wanted make genocide, believe me you wouldn’t find one alive person in Karabagh.ıf Turkey decides to enter war with Armenia, in 24 hours we controll all over the Armenia\n#NewYork voters are calling on @RepAOC to sign H.Res.1165 this week, ensuring that U.S. tax dollars are not funding Azerbaijan war 

2310

In [25]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 100

# Delete any paragraphs shorter than MIN_LEN_PARAG
text_parags_ls = [x for x in text_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
text_parags_ls = [' '.join(x.split()) for x in text_parags_ls]

len(text_parags_ls)

2999

In [26]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars
"""
parag_ct = len(text_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = text_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')
"""

"\nparag_ct = len(text_parags_ls)\n\ndoc_now_str = ''\ndoc_now_len = 0\ndocs = []\n\nfor i in range(parag_ct):\n  # print(f'Processing Paragraph #{i}')\n  parag_now_str = text_parags_ls[i]\n  doc_now_str += parag_now_str\n  doc_now_len += len(parag_now_str)\n  if doc_now_len > MIN_LEN_DOC:\n    docs.append(doc_now_str)\n    doc_now_str = ''\n    doc_now_len = 0\n\ndocs[-1] += doc_now_str\n\nprint(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')\n"

In [27]:
# View the first 5 docs

doc_ls[:5]

['End this war. Azerbaijan military bombed the Maternity Hospital in Stepanakert #Artsakh #NagornoKarabakh https://t.co/2aF8Nq1Vhm\nPeople of Artsakh are Azerbaijan’s target – Armenian MFA voices about war crimes https://t.co/pPiXAY1FyM\nThis war crime, which is a gross violation of international humanitarian law and customary law, clearly demonstrates that the people of Artsakh are Azerbaijan’s target, including newborn babies, mothers, and the elders.\n#StopAzerbaijaniAggerssion https://t.co/7kn7uBf2uI\nPeople of Artsakh are Azerbaijan’s target – Armenian MFA voices about war crimes https://t.co/cu3mLr73lu\nArmenia want peace❤️💙🧡\nThe war in Artsakh was started by Azerbaijan together with Turkey, it is violent war, they are bombing civilians.\nP. S. Open your eyes and stop Genocide Politics of Azerbaijan and Turkey. \n #PeaceForArmenians #StopAzerbaijaniTerrorism\n@deanbaquet\xa0@carlottagall\xa0I demand a rewrite for this irresponsible and misinformed message. You completely omitted

In [28]:
# View the first 500 chars in the 50th doc

doc_ls[49][:500]

'Azerbaijan under Turkey’s sponsorship started a war against Artsakh! They target peaceful civilians with internationally prohibited weapons! Stop this war! #BoycottTurkey #BoycottTurquie https://t.co/nGNDCdzxTU\nShame on\xa0@NYTimes\xa0for \xa0misreporting on the war that Azerbaijan started in Artsakh. You are responsible for the continuous\xa0#AzeriWarCrimes committed on innocent civilians in Artsakh. #PeaceforArmenians\n@SenatorHousakos Thank you @SenatorHousakos  for standing up for the peaceful population'

In [29]:
# View the last 100 chars in the last doc

doc_ls[-1][-100:]

'NagornoKarabakh #Syunik #Kapan #ArtsakhBlockade #Armenia #Azerbaijan @5050oD\nhttps://t.co/3w1pNgyOdr'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Steps

#### Expand Contractions

In [32]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
contractions.fix("yall're happy now it isn't a don't tell problem", slang=False) # default: true

"yall're happy now it is not a do not tell problem"

In [33]:
doc_ls[:5]

['End this war. Azerbaijan military bombed the Maternity Hospital in Stepanakert #Artsakh #NagornoKarabakh https://t.co/2aF8Nq1Vhm\nPeople of Artsakh are Azerbaijan’s target – Armenian MFA voices about war crimes https://t.co/pPiXAY1FyM\nThis war crime, which is a gross violation of international humanitarian law and customary law, clearly demonstrates that the people of Artsakh are Azerbaijan’s target, including newborn babies, mothers, and the elders.\n#StopAzerbaijaniAggerssion https://t.co/7kn7uBf2uI\nPeople of Artsakh are Azerbaijan’s target – Armenian MFA voices about war crimes https://t.co/cu3mLr73lu\nArmenia want peace❤️💙🧡\nThe war in Artsakh was started by Azerbaijan together with Turkey, it is violent war, they are bombing civilians.\nP. S. Open your eyes and stop Genocide Politics of Azerbaijan and Turkey. \n #PeaceForArmenians #StopAzerbaijaniTerrorism\n@deanbaquet\xa0@carlottagall\xa0I demand a rewrite for this irresponsible and misinformed message. You completely omitted

In [34]:
len(doc_ls)

2999

In [35]:
# Expand all Contractions paragraph by paragraph

"""
docs_clean_ls = []
for adoc in doc_ls:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]
""";

docs = doc_ls

'\ndocs_clean_ls = []\nfor adoc in doc_ls:\n  docs_clean_ls.append(contractions.fix(adoc))\n\ndocs = docs_clean_ls\ndocs[:5]\n'

#### Tokenize Text

In [36]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [37]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

['end',
 'this',
 'war',
 'azerbaijan',
 'military',
 'bombed',
 'the',
 'maternity',
 'hospital',
 'in',
 'stepanakert',
 'artsakh',
 'nagornokarabakh',
 'https',
 'co',
 '2af8nq1vhm',
 'people',
 'of',
 'artsakh',
 'are',
 'azerbaijan',
 'target',
 'armenian',
 'mfa',
 'voices',
 'about',
 'war',
 'crimes',
 'https',
 'co',
 'ppixay1fym',
 'this',
 'war',
 'crime',
 'which',
 'is',
 'gross',
 'violation',
 'of',
 'international',
 'humanitarian',
 'law',
 'and',
 'customary',
 'law',
 'clearly',
 'demonstrates',
 'that',
 'the',
 'people',
 'of',
 'artsakh',
 'are',
 'azerbaijan',
 'target',
 'including',
 'newborn',
 'babies',
 'mothers',
 'and',
 'the',
 'elders',
 'stopazerbaijaniaggerssion',
 'https',
 'co',
 '7kn7ubf2ui',
 'people',
 'of',
 'artsakh',
 'are',
 'azerbaijan',
 'target',
 'armenian',
 'mfa',
 'voices',
 'about',
 'war',
 'crimes',
 'https',
 'co',
 'cu3mlr73lu',
 'armenia',
 'want',
 'peace',
 'the',
 'war',
 'in',
 'artsakh',
 'was',
 'started',
 'by',
 'azerbaija



There are 234 tokens in the first document


#### Customize Stopwords

In [38]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [39]:
# Define list of stopwords listed above that we should keep

stopword_keep_ls = []

"""
stopword_keep_ls = [
    'against',
    'not',
    'no',
    'nor',

    'do',
    'are',
    'is',
    'are',
    "not",
    'could',
    'did',
    'have',
    'has',
    'had',
    'might',
    'must',
    'need',
    'shall',
    'should',
    'was',
    'were',
    'would',

    'own',
    'same',
    'how',
    'why'
]
"""

'\nstopword_keep_ls = [\n    \'against\',\n    \'not\',\n    \'no\',\n    \'nor\',\n\n    \'do\',\n    \'are\',\n    \'is\',\n    \'are\',\n    "not",\n    \'could\',\n    \'did\',\n    \'have\',\n    \'has\',\n    \'had\',\n    \'might\',\n    \'must\',\n    \'need\',\n    \'shall\',\n    \'should\',\n    \'was\',\n    \'were\',\n    \'would\',\n\n    \'own\',\n    \'same\',\n    \'how\',\n    \'why\'\n]\n'

In [40]:
# Derive final stopword list

stopword_keep_ls = list(set(stopwords_ls) - set(stopword_keep_ls))
stopword_keep_ls
print('\n\n')
print(f'BEFORE: {len(stopwords_ls)} stopwords')
print(f' AFTER: {len(stopword_keep_ls)}')

['to',
 'between',
 'up',
 'herself',
 'themselves',
 "shouldn't",
 'himself',
 'can',
 'me',
 'did',
 'ourselves',
 'but',
 'am',
 "needn't",
 'his',
 'been',
 'should',
 'don',
 'ma',
 "isn't",
 'our',
 'we',
 'the',
 'wasn',
 'theirs',
 'have',
 'ain',
 'being',
 'they',
 'shan',
 "wouldn't",
 "should've",
 'now',
 'o',
 'her',
 'those',
 "shan't",
 'there',
 'shouldn',
 'own',
 'nor',
 'about',
 'through',
 'in',
 'as',
 'under',
 "wasn't",
 'doing',
 'few',
 'your',
 'each',
 "won't",
 'any',
 'very',
 'until',
 "hadn't",
 'other',
 'whom',
 "you're",
 "don't",
 'yourselves',
 'from',
 'all',
 'or',
 'over',
 "haven't",
 "mustn't",
 'not',
 'has',
 'before',
 'isn',
 've',
 'weren',
 'do',
 'down',
 'then',
 'into',
 'a',
 'than',
 "mightn't",
 'during',
 "that'll",
 't',
 'hasn',
 'mightn',
 'mustn',
 'myself',
 'with',
 'why',
 'no',
 'll',
 "weren't",
 'of',
 'are',
 'for',
 're',
 'out',
 'itself',
 'its',
 'you',
 'while',
 'above',
 "you'd",
 'were',
 'aren',
 'haven',
 'tha




BEFORE: 179 stopwords
 AFTER: 179


In [41]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopword_keep_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')


['wasn',
 'theirs',
 'have',
 'ain',
 'being',
 'they',
 'shan',
 'now',
 'o',
 'her',
 'those',
 'there',
 'shouldn',
 'own',
 'nor',
 'about',
 'through',
 'in',
 'as',
 'under',
 'doing',
 'few',
 'your',
 'each',
 'any',
 'very',
 'until',
 'other',
 'whom',
 'yourselves',
 'from',
 'all',
 'or',
 'over',
 'not',
 'has',
 'before',
 'isn',
 've',
 'weren',
 'do',
 'down',
 'then',
 'into',
 'a',
 'than',
 'during',
 't',
 'hasn',
 'mightn',
 'mustn',
 'myself',
 'with',
 'why',
 'no',
 'll',
 'of',
 'are',
 'for',
 're',
 'out',
 'itself',
 'its',
 'you',
 'while',
 'above',
 'were',
 'aren',
 'haven',
 'that',
 'how',
 'here',
 'was',
 'below',
 'most',
 'won',
 'will',
 'these',
 'against',
 'only',
 'hers',
 'yours',
 'both',
 'ours',
 'hadn',
 'it',
 'where',
 'some',
 'wouldn',
 'needn',
 'after',
 'having',
 'this',
 'doesn',
 'didn',
 'so',
 'm',
 'off',
 'who',
 'couldn',
 'once',
 'him',
 'which',
 's',
 'does',
 'an',
 'again',
 'my',
 'their',
 'y',
 'further',
 'just',




There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [42]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


#### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [43]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [44]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 6.4 s, sys: 239 ms, total: 6.64 s
Wall time: 6.7 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [45]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [46]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 5 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.49 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [47]:
# docs_ls_ls = [astring.split() for astring in docs]
# docs = docs_ls_ls

In [48]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [49]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [50]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

2999

In [51]:
type(corpus[0])

list

In [52]:
corpus[0][:10]

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2)]

In [53]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 8769
Number of documents: 2999


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [30]:
#@markdown How many Topics do you want to find?

No_of_Topics = 10 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [54]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 1min 14s, sys: 651 ms, total: 1min 15s
Wall time: 1min 15s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [55]:
%whos str

Variable            Type    Data/Info
-------------------------------------
TEXT_ENCODING       str     utf-8
Text_Title          str       [artsakhwar] from [20201028] to [20221214]
aline               str     @sovorakan_ton Cutting of<...>nce and it is running out
doc_temp_str        str     @Melmouth1 @AvetissianAn <...>\nhttps://t.co/3w1pNgyOdr
end_date            str     20221214
end_time            str     195302.csv
fn                  str     tweets_raw_artsakhwar_202<...>45527_20221214_195302.csv
search_str          str     artsakhwar
start_date          str     20201028
start_time          str     145527
temp                str     almost
text_filename_str   str     tweets_raw_artsakhwar_202<...>45527_20221214_195302.csv
text_raw_str        str     End this war. Azerbaijan <...>\nhttps://t.co/3w1pNgyOdr
text_title_str      str     Artsakhwar From 20201028 To 20221214
token               str     3w1pngyodr
word1               str     tweets
word2               str     raw


In [56]:
search_str

'artsakhwar'

In [57]:
top_topics = model.top_topics(corpus) #, num_words=20)


# from pprint import pprint
# pprint(top_topics)

# print('\n\n')
# for i,atopic in enumerate(top_topics):
#   atopic_coherence_str = f'Topic #{i}: coherence = {atopic[1]}'
#   print(atopic_coherence_str)
#   report_line_ls.append(atopic_coherence_str)


topic_word_ls = []

topic_word_ls.append(text_title_str)

# print(text_title_str)

for i, atopic in enumerate(top_topics):
  topic_str = f'\n\nTopic #{i}: {atopic[1]} (coherence)'
  # print(topic_str)
  topic_word_ls.append(topic_str)
  for j, aword in enumerate(atopic[0]):
    word_str = f'  Word #{j}: {aword}'
    # print(word_str)
    topic_word_ls.append(word_str)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
avg_topic_coherence_str = f'Average topic coherence: {avg_topic_coherence:.4f}'
# print(avg_topic_coherence_str)
topic_word_ls.append(f'\n\n{avg_topic_coherence_str}')

topic_word_str = "\n".join(topic_word_ls)
print(topic_word_str)

# Save to file
filename_topic_word_str = f'lda_topic_{search_str}_word_report.txt'
with open(filename_topic_word_str, 'w') as fp:
  fp.write(topic_word_str)

files.download(filename_topic_word_str)

Artsakhwar From 20201028 To 20221214


Topic #0: -0.8222181044094025 (coherence)
  Word #0: (0.0059585697, 'were')
  Word #1: (0.005910319, 'ukraine')
  Word #2: (0.0054058344, 'russia')
  Word #3: (0.0051422054, 'when')
  Word #4: (0.0050677666, 'after')
  Word #5: (0.004925332, 'azeri')
  Word #6: (0.004622802, 'your')
  Word #7: (0.004618704, 'aliyev')
  Word #8: (0.0045937877, 'no')
  Word #9: (0.00444499, 'there')
  Word #10: (0.0043765595, 'now')
  Word #11: (0.004319381, 'year')
  Word #12: (0.0041719247, 'one')
  Word #13: (0.004090098, 'he')
  Word #14: (0.0039598388, 'if')
  Word #15: (0.0038999005, 'azerbaijani')
  Word #16: (0.0038608406, 'first')
  Word #17: (0.0037703703, 'just')
  Word #18: (0.0037230458, 'at')
  Word #19: (0.003580759, 'or')


Topic #1: -1.0251928622726973 (coherence)
  Word #0: (0.007051551, 'our')
  Word #1: (0.0062487912, 'were')
  Word #2: (0.006239425, 'after')
  Word #3: (0.0062162247, 'he')
  Word #4: (0.0060782963, 'year')
  Word #5: (0.00541762

7906

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize

In [58]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [59]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


